In [14]:
import minsearch
import json

In [15]:
with open("documents.json", 'rt') as f_in:
    docs_raw = json.load(f_in)

In [16]:
documents = []

# Flatten the documents
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [17]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [18]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"] # SELECT * WHERE course = 'data-engineering-zoomcamp'; #kind of SQL query for keyword_fields
)

In [19]:
q = "the course has already started, can i still enroll?" #question example

In [20]:
index.fit(documents)

In [21]:
# All these fields como from elastic search and buy default they are all equal to 1.0

boost = {
    "question": 3.0, #the question field is 3 times mr important than the other fields
    'section':0.5 # this filed is 0.5 times less important than the other fields
}

results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'}, #filter for a specific course
    boost_dict=boost,
    num_results=5
)

In [22]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [23]:
from openai import OpenAI
import os

In [24]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [25]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{
        "role": "user", 
        "content": "is it too late to join the course?"
         },
    ]
)

In [26]:
response.choices[0].message.content

"The timing for joining a course can depend on several factors, including the specific course policies, the institution offering the course, and how far into the course it currently is. Here are a few steps you can take to determine if it's too late to join:\n\n1. **Check Course Deadlines**: Look for any published deadlines for registration or late enrollment. Many institutions have clear cutoff dates for joining courses.\n\n2. **Contact the Instructor or Institution**: Reach out directly to the course instructor or the admissions office. Sometimes, exceptions can be made if you have a strong reason for joining late.\n\n3. **Review the Syllabus**: If you have access, review the course syllabus to understand what you might have missed and whether you can realistically catch up on the material.\n\n4. **Consider the Coursework**: Evaluate whether you have the time and resources to catch up on any missed assignments, readings, or lectures.\n\n5. **Policy on Late Enrollment**: Some institut

In [27]:
prompt_template = """
You're a course teaching assistant.Answer the QUESTION based on CONTEXT from the FAQ database.
Use only the facts from CONTEXT when answering when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT:{context}

""".strip()

In [28]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [29]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start wit

In [30]:
prompt = prompt_template.format(question=q, context=context).strip()

In [31]:
print(prompt)

You're a course teaching assistant.Answer the QUESTION based on CONTEXT from the FAQ database.
Use only the facts from CONTEXT when answering when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: the course has already started, can i still enroll?

CONTEXT:section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on you

In [32]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{
        "role": "user", 
        "content": prompt
         },
    ]
)

In [33]:
print(response.choices[0].message.content)

Yes, you can still join the course after it has started. Even if you don't register, you're eligible to submit the homeworks. However, be aware of the deadlines for turning in the final projects.
